In [36]:
import os
import imageio

os.environ['CUDA_VISIBLE_DEVICES'] = '15'

from tcvae.models import TCVAE
from tcvae.data import ImageDataGenerator
from tcvae.utils import import_project_root, make_directory
from tcvae.visualization import process_for_animation, animate_traversals

import_project_root()
from config import EXPERIMENTS_DIR, CELEB_A_DIR

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
EXPERIMENT_NAME = 'beta-vae-sse-8'
NUM_IMAGES = 25
NUM_ROWS = 5
STD_THRESHOLD = 0.75
OVERWRITE = True

In [31]:
experiment_dir = EXPERIMENTS_DIR / EXPERIMENT_NAME
model_dir = experiment_dir / 'model'
traversal_dir = experiment_dir / 'traversals'
make_directory(traversal_dir, overwrite=OVERWRITE)

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
model = TCVAE.load(model_dir)
datagen = ImageDataGenerator(CELEB_A_DIR, NUM_IMAGES)
imgs = datagen.load_n_images(NUM_IMAGES)

In [10]:
traversals = model.make_all_traversals(
    imgs, num_rows=NUM_ROWS, std_threshold=STD_THRESHOLD)
for key, value in traversals.items():
    traversals[key] = process_for_animation(value)

Generating traversals for the following 27 latent indices:
[ 1  2  3  4  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 25 26 27 28
 29 30 31]


# As script

In [54]:
%%writefile '/project/scripts/02-animate-traversals.py'

import os
import argparse as ap

import imageio

from tcvae.models import TCVAE
from tcvae.data import ImageDataGenerator
from tcvae.utils import import_project_root, make_directory
from tcvae.visualization import process_for_animation, animate_traversals

import_project_root()
from config import EXPERIMENTS_DIR, CELEB_A_DIR


def parse_args():
    parser = ap.ArgumentParser()
    parser.add_argument(
        'experiment_name', type=str, help=(
            'The base name of the directory containing resources for a specific '
            'experiment.'))
    parser.add_argument(
        '-i', '--num_imgs', type=int, default=25,
        help='The number of images on which to perform traversals.')
    parser.add_argument(
        '-r', '--num_rows', type=int, default=5,
        help='The number of rows to use when tiling images together.')
    parser.add_argument(
        '-t', '--std_threshold', type=float, default=0.8,
        help=(
            'Latent dimensions with average standard deviations above this '
            'threshold are not included in traversals.'))
    parser.add_argument(
        '-o', '--overwrite', action='store_true',
        help='If true, overwrites directory containing existing traversals.')
    parser.add_argument(
        '-g', '--gpu', type=int, default=15,
        help='The ID of the GPU to use for generating traversals.')
    args = parser.parse_args()
    return args
    

def main():
    
    args = parse_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu)
    
    experiment_dir = EXPERIMENTS_DIR / args.experiment_name
    model_dir = experiment_dir / 'model'
    traversal_dir = experiment_dir / 'traversals'
    make_directory(traversal_dir, overwrite=args.overwrite)
    
    model = TCVAE.load(model_dir)
    datagen = ImageDataGenerator(CELEB_A_DIR, args.num_imgs)
    imgs = datagen.load_n_images(args.num_imgs)
    
    traversals = model.make_all_traversals(
        imgs, num_rows=args.num_rows, std_threshold=args.std_threshold)
    for key, value in traversals.items():
        traversals[key] = process_for_animation(value)
        
    animate_traversals(traversals, traversal_dir)


if __name__ == '__main__':
    main()

Overwriting /project/scripts/02-animate-traversals.py
